# Cerebral Cortex Data Analysis Algorithms
Cerebral Cortex contains a library of algorithms that are useful for processing data and converting it into features or biomarkers.  This page demonstrates a simple GPS clustering algorithm.  For more details about the algorithms that are available, please see our [documentation](https://cerebralcortex-kernel.readthedocs.io/en/latest/).  These algorithms are constantly being developed and improved through our own work and the work of other researchers.

## Initalize the system

In [1]:
%reload_ext autoreload
from util.dependencies import *

CC = Kernel("/home/md2k/cc_conf/", study_name="default")

## Generate some sample location data

This example utilizes a data generator to protect the privacy of real participants and allows for anyone utilizing this system to explore the data without required institutional review board approvals. This is disabled for this demonstration to not create too much data at once.

In [2]:
gen_location_datastream(CC, user_id=USER_ID, stream_name="GPS--org.md2k.phonesensor--PHONE")

## Get stream data
Read the demo GPS stream and show some example values.  A typical GPS sample contains values for _latitude, longitude, altitude, speed, bearing, and accuracy_.

In [3]:
gps_stream = CC.get_stream("GPS--org.md2k.phonesensor--PHONE")
gps_stream.show(3)

+-------------------+-------------------+------------------+------------------+--------+--------+----------+---------+-------+--------------------+
|          timestamp|          localtime|          latitude|         longitude|altitude|   speed|   bearing| accuracy|version|                user|
+-------------------+-------------------+------------------+------------------+--------+--------+----------+---------+-------+--------------------+
|2019-09-01 22:26:59|2019-09-02 03:26:59|35.157442557926856| -90.0349952849216|      93|2.834021|105.992312|17.288347|      1|00000000-afb8-476...|
|2019-09-01 22:27:59|2019-09-02 03:27:59|  35.1561572521825|-90.03681018833898|      94|3.796146|  8.256657|15.215249|      1|00000000-afb8-476...|
|2019-09-01 22:28:59|2019-09-02 03:28:59| 35.15747745895255|-90.03574140915266|      93| 1.11443|   4.37211|11.172609|      1|00000000-afb8-476...|
+-------------------+-------------------+------------------+------------------+--------+--------+----------+----

## Cluster the location data
Cerebral Cortex makes it easy to apply built-in algorithms to data streams.  In this case, `gps_clusters` is imported from the algorithm library, then `compute` is utilized to run this algorithm on the `gps_stream` to generate a set of centroids. This is the general format for applying algorithm to datastream and makes it easy for researchers to apply validated and tested algorithms to his/her own data without the need to become an expert in the particular set of transformations needed.  

_Note:_ the `compute` method engages the parallel computation capabilities of Cerebral Cortex, which causes all the data to be read from the data storage layer and processed on every computational core available to the system.  This allows the computation to run as quickly as possible and to take advantage of powerful clusters from a relatively simple interface.  This capability is critical to working with mobile sensor big data where data sizes can exceed 100s of gigabytes per datastream for larger studies.

In [4]:
from cerebralcortex.algorithms.gps.clustering import cluster_gps
windowed_gps = gps_stream.window()
clusters = cluster_gps(windowed_gps)
clusters.show(truncate=False)

+-------------------+-------------------+------------------+------------------+--------+--------+----------+---------+-------+------------------------------------+------------------+------------------+-----------+------------------+
|timestamp          |localtime          |latitude          |longitude         |altitude|speed   |bearing   |accuracy |version|user                                |centroid_longitude|centroid_latitude |centroid_id|centroid_area     |
+-------------------+-------------------+------------------+------------------+--------+--------+----------+---------+-------+------------------------------------+------------------+------------------+-----------+------------------+
|2019-09-01 22:26:59|2019-09-02 03:26:59|35.157442557926856|-90.0349952849216 |93      |2.834021|105.992312|17.288347|1      |00000000-afb8-476e-9872-6472b4e66b68|-90.0353360730015 |35.157759542552085|0          |267.12246418031026|
|2019-09-01 22:27:59|2019-09-02 03:27:59|35.1561572521825  |-90.0368

## Visualize GPS Data

### GPS Stream Plot
GPS visualization requires dedicated plotting capabilities. Cerebral Cortex includes a library to allow for interactive exploration.  In this plot, use your mouse to drag the map around along with zooming in to explore the specific data points.

In [5]:
from cerebralcortex.plotting.gps.plots import plot_gps_clusters
plot_gps_clusters(clusters,zoom=8)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …